In [3]:
import pandas as pd
import numpy as np
import sys
sys.path.append("/Users/derekdewald/Documents/Python/Github_Repo/d_py_functions/")

from Organization import CreateMarkdownfromProcess

CreateMarkdownfromProcess()

